#Display sample images & draw truth labels to visualize data from tfrecord files.

##Imports.

In [1]:
from PIL import Image
import io
import numpy as np
import tensorflow as tf

##Read tfrcord files.

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/facial_landmark_pred_example/tfrec_data/'

raw_dataset = tf.data.TFRecordDataset([path + 'tfrec_data_1', 
                                       path + 'tfrec_data_2',
                                       path + 'tfrec_data_3', 
                                       path + 'tfrec_data_4', 
                                       path + 'tfrec_data_5', 
                                       ])

print(raw_dataset)
print(raw_dataset.cardinality())
print(tf.data.experimental.cardinality(raw_dataset).numpy())
cardinality = tf.data.experimental.cardinality(raw_dataset)
print((cardinality == tf.data.experimental.INFINITE_CARDINALITY).numpy())
print((cardinality == tf.data.experimental.UNKNOWN_CARDINALITY).numpy())
len(list(raw_dataset))

In [ ]:
for raw_record in raw_dataset.take(3):
    print(repr(raw_record))

In [4]:
# Create a description of the features.
feature_description = {
    'img_name': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'img': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'coords': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

In [ ]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

###Test samples by displaying some images.

In [ ]:
n=3     # Number of samples.
for i, parsed_record in enumerate(parsed_dataset.take(n)):
    img_name = parsed_record['img_name'].numpy().decode('UTF-8')
    print("{} {}".format(i, img_name))
    img = parsed_record['img'].numpy()

    pic = Image.open(io.BytesIO(img))
    display(pic)

    coords = np.frombuffer(parsed_record['coords'].numpy())

###Draw truth labels on images.
 1. The pixels drawn are very small, click to zoom in to verify the overlay pixels.

In [ ]:
coords = coords.reshape(194,2)

for coord in coords:    
    x = coord[0] 
    y = coord[1] 
    pic.putpixel( (round(x), round(y)), (0, 255, 0, 0) )

 # The pixels drawn are very small,
 # click to zoom in to verify the overlay pixels.
display(pic)     

###Resize pic & scale coordinates to check pixels overlay.

In [ ]:
scale_ratio = 0.25      # Scale while preserving aspect ratio.
width = int(pic.size[0] * scale_ratio)
height = int(pic.size[1] * scale_ratio)    
dim = (width, height)
pic_resize = pic.resize(dim, resample=0, box=None)
coords_resize = coords * scale_ratio      # Scale the coordinates.

for coord in coords_resize:    
    x = coord[0] 
    y = coord[1] 
    pic_resize.putpixel( (round(x), round(y)), (255, 0, 0) )

display(pic_resize)     